## **示例：用于预订酒店的多AI代理**

在当今快节奏的世界中，规划商务旅行不仅仅是预订航班和酒店房间。它需要一种协调和效率的水平，这可能很难实现。这就是多AI代理发挥作用的地方，彻底改变了我们管理旅行需求的方式。

想象一下，有一支智能代理团队随时为您服务，协同处理旅行的各个方面，做到精准且轻松。借助我们先进的AI技术，我们创建了专门负责预订服务和行程安排的代理，确保无缝且无压力的旅行体验。

这是一个基本场景。在规划商务旅行时，我们需要咨询商务旅行代理以获取机票信息、酒店信息等。通过AI代理，我们可以构建负责预订服务的代理和负责行程安排的代理进行协作，提高智能化水平。


# 初始化 Azure AI Agent 服务并从 **.env** 获取配置信息

### **.env**

创建一个 .env 文件

**.env** 包含 Azure AI Agent 服务的连接字符串、AOAI 使用的模型，以及相应的 Google API 搜索服务 API、ENDPOINT 等。

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "您的 Azure AI Agent 服务模型部署名称"

[**注意**] 您需要一个具有 100,000 速率限制（每分钟令牌数）以及 600 速率限制（每分钟请求数）的模型

  您可以在 Microsoft Foundry - 模型和端点中获取模型。

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "您的 Azure AI Agent 服务项目连接字符串"

  您可以在 AI Foundry 门户屏幕的项目概览中获取项目连接字符串。

- **SERPAPI_SEARCH_API_KEY** = "您的 SERPAPI 搜索 API 密钥"
- **SERPAPI_SEARCH_ENDPOINT** = "您的 SERPAPI 搜索端点"

要获取 Azure AI Agent 服务的模型部署名称和项目连接字符串，您需要创建 Azure AI Agent 服务。建议使用[此模板](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json)直接创建（***注意：*** Azure AI Agent 服务目前仅在有限区域设置。建议您参考[此链接](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support)设置区域）

Agent 需要访问 SERPAPI。建议使用[此链接](https://serpapi.com/searches)注册。注册后，您可以获得唯一的 API KEY 和 ENDPOINT。


# 登录到 Azure

您现在需要登录到 Azure。 在 VScode 中打开终端并运行 `az login` 命令。


# Setup 

要运行此笔记本，您需要安装以下库。以下是所需库及相应的 pip 安装命令列表：

azure-identity：用于 Azure 认证。  
requests：用于发起 HTTP 请求。  
semantic-kernel：用于语义内核框架（假设这是一个自定义或特定的库，您可能需要从特定源或仓库进行安装）。


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Explanation: 
import asyncio: 这将导入 asyncio 模块，该模块为 Python 中的异步编程提供支持。它允许你使用 async 和 await 语法编写并发代码。
from typing import Annotated: 这将从 typing 模块导入 Annotated 类型。Annotated 用于向类型提示添加元数据，这在验证、文档或工具等各种场景中都非常有用。


In [ ]:
import asyncio,os
from typing import Annotated

# Explanation:
通过使用 from dotenv import load_dotenv 和 load_dotenv()，您可以轻松地在 .env 文件中管理配置设置和敏感信息（如 API 密钥和数据库 URL），将它们与源代码分离，使您的应用程序更安全且更易于配置。


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# 说明：

导入语句：from azure.identity.aio import DefaultAzureCredential：这会从 azure.identity.aio 模块导入 DefaultAzureCredential 类。模块名称中的 aio 部分表示该模块是为异步操作设计的。

DefaultAzureCredential 的用途：DefaultAzureCredential 类是 Python 版 Azure SDK 的一部分。它提供了一种默认的方式来对 Azure 服务进行身份验证。它会按特定顺序尝试使用多种方法进行身份验证，例如环境变量、托管身份和 Azure CLI 凭据。

异步操作：aio 模块表示 DefaultAzureCredential 类支持异步操作。这意味着您可以使用 asyncio 来执行非阻塞的身份验证请求。


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# 解释：
从 semantic_kernel 包中导入了各种模块和类。以下是每个导入的详细说明：

AgentGroupChat 来自 semantic_kernel.agents：该类处理与 AI 代理的群聊相关的功能。  
AzureAIAgent 和 AzureAIAgentSettings 来自 semantic_kernel.agents.azure_ai：

AzureAIAgent：该类用于创建和管理利用 Azure AI 服务的 AI 代理。

AzureAIAgentSettings：该类用于配置 AzureAIAgent 的设置。  
TerminationStrategy 来自 semantic_kernel.agents.strategies.termination.termination_strategy：

该类定义了在特定条件下终止 AI 代理执行的策略。  
ChatMessageContent 来自 semantic_kernel.contents.chat_message_content：

该类用于处理聊天消息的内容。  
AuthorRole 来自 semantic_kernel.contents.utils.author_role：

该类定义了聊天消息中作者的不同角色。  

kernel_function 来自 semantic_kernel.functions.kernel_function_decorator：该装饰器用于定义内核函数，即可以在语义内核框架内执行的函数。  
这些导入为创建和管理能够在群聊环境中交互的 AI 代理设置了必要组件，这些代理可能执行诸如预订酒店或类似任务。


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# 解释：
接下来我们从 azure.ai.projects.models 模块导入 CodeInterpreterTool 类。

CodeInterpreterTool：此类是 Azure AI SDK 的一部分，用于在 AI 项目的上下文中解释和执行代码。它提供运行代码片段、分析代码或将代码执行集成到 AI 工作流中的功能。
此导入为在项目中使用 CodeInterpreterTool 配置了必要的组件，这对于涉及动态解释和执行代码的任务非常有用。


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# 解释：
ApprovalTerminationStrategy 类提供了一种特定的策略，用于终止 AI 代理的操作。如果代理交互历史中的最后一条消息包含“saved”一词，则代理将终止。这在代理任务一旦收到“saved”确认即视为完成的场景中可能很有用。定义交互方法。在保存预订计划后，在接收到 saved 信号时可以停止。


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# 说明：

该代码行通过调用 create() 方法初始化一个具有默认或预定义设置的 AzureAIAgentSettings 对象。该设置对象（ai_agent_settings）随后可用于配置和管理 AzureAIAgent 实例。


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# 说明：
通过导入 requests 库，您可以轻松地在 Python 代码中发出 HTTP 请求并与网络服务进行交互。


In [ ]:
import requests

# 说明：
这是一个用于存储访问 SERP（搜索引擎结果页面）API 服务的 API 密钥的变量。API 密钥是用于验证与您的帐户相关请求的唯一标识符。

'GOOGLE_SEARCH_API_KEY'：这是一个占位符字符串。您需要用实际的 SERP API 密钥替换 ''GOOGLE_SEARCH_API_KEY'。

用途：这行代码的目的是将 API 密钥存储在变量中，以便用于验证对 SERP API 服务的请求。访问该服务并执行搜索需要提供 API 密钥。

如何获取 SERP API 密钥：要获取 SERP API 密钥，请按照 https://serpapi.com 上的一般步骤操作（具体步骤可能因所使用的特定 SERP API 服务而异）：

选择一个 SERP API 服务：有几个 SERP API 服务可供选择，如 SerpAPI、Google 自定义搜索 JSON API 等。请选择最适合您需求的服务。

注册账户：

访问所选 SERP API 服务的网站 https://www.serpapi.com 并注册账户。您可能需要提供一些基本信息并验证您的电子邮件地址。

创建 API 密钥：

注册后，登录您的账户并导航到 API 部分或仪表盘。查找创建或生成新 API 密钥的选项。
复制 API 密钥：

生成 API 密钥后，复制该密钥。此密钥将用于验证您对 SERP API 服务的请求。
替换占位符：

用实际密钥替换您 .env 文件中的占位符。


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# 说明：
BookingPlugin 类提供了使用 Serpapi.com Google 搜索 API 预订酒店和机票的方法。它构建所需的参数，发送 API 请求，并处理响应以返回相关的预订信息。API 密钥（SERPAPI_SEARCH_API_KEY）和端点（SERPAPI_SEARCH_ENDPOINT）用于身份验证并向 Google 搜索 API 发送请求。


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# 说明：
SavePlugin 类提供了一个 saving_plan 方法，用于使用 Azure AI 服务保存旅行计划。它设置了 Azure 凭证，创建了一个 AI 代理，处理用户输入以生成并保存旅行计划内容，并处理文件保存和清理操作。该方法在成功完成后返回“Saved”。


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# 说明：
此代码设置了 Azure AI 代理来处理航班和酒店预订，以及根据用户输入保存旅行计划。它使用 Azure 凭据来创建和配置代理，通过群聊处理用户输入，并确保任务完成后进行适当的清理。代理使用特定的插件（BookingPlugin 和 SavePlugin）来执行各自的任务。


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免责声明**：  
本文件使用 AI 翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 进行翻译。虽然我们力求准确，但请注意，自动翻译可能包含错误或不准确之处。应以原始语言的文件为权威来源。对于重要信息，建议寻求专业人工翻译。因使用此翻译而产生的任何误解或误释，我们不承担任何责任。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
